# Guided Project #

This is a first guided project of SQL in dataquest, an online platform where you can learn data analysis and science. Guided project is a project where problems are provided and I try to solve them using what I have learned. 

In this project, I solved business questions using Sqlite3. A sample database I worked on is Chinook database. The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables. The access to the database from [here](https://github.com/lerocha/chinook-database)

__4 tasks to do__
- selecting albums to purchase from artsts 
- analyzing employee's performance 
- analyzing sales by country 
- analyzing album purchases and individual tracks purchases. 



## Connecting this notebook to our database file 

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Get familiar with a database and its tables ##

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type in ("table","view")

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Task 1 ##
The context I was given is here. The text below is directly quoted from dataquest page. 

"The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

Artist Name 
- Regal 
- Red Tone
- Meteor and the Girls
- Slim Jim Bites

Genre(respectably)
- Hip-Hop
- Punk
- Pop
- Blues

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.
You'll need to write a query to find out which genres sell the most tracks in the USA, write up a summary of your findings, and make a recommendation for the three artists whose albums we should purchase for the store."


In [3]:
%%sql
WITH country_genre AS
(
    SELECT 
        c.country country,
        t.track_id track_id,
        g.name genre
    FROM customer c 
        INNER JOIN invoice i on c.customer_id = i.customer_id
        INNER JOIN invoice_line il on i.invoice_id = il.invoice_id
        INNER JOIN track t on il.track_id = t.track_id
        INNER JOIN genre g on g.genre_id = t.genre_id
        WHERE c.country ="USA"
)
SELECT 
    genre,COUNT(track_id) tracks_sold,
    CAST(COUNT(track_id) AS float) / (select COUNT(*) From country_genre) percentage
FROM country_genre
GROUP BY genre
ORDER BY 3 DESC


Done.


genre,tracks_sold,percentage
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


In America, Rock, Alternative & Punk, and Metal are top 3 popular music genres.
From the list of artists, we should purchase albums of 'Red Tone', 'Slim Jim Bites', and 'Meteor and te Girls' if we have to pick three artists from the list. This is because, their genres are popular in the list. 

## Task 2
"Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.
You might like to consider whether any extra columns from the employee table explain any variance you see, or whether the variance might instead be indicative of employee performance."(direct quote from dataquest page) 

"Write a query that finds the total dollar amount of sales assigned to each sales support agent within the company. Add any extra attributes for that employee that you find are relevant to the analysis."(direct quote from dataquest page) 

In [4]:
%%sql
CREATE VIEW sales_agent AS
SELECT 
    e.employee_id,
    e.first_name ||" "|| e.last_name name, 
    e.title, 
    e.hire_date,
    i.total total,
    i.invoice_date date
FROM employee e
    LEFT JOIN customer c ON c.support_rep_id = e.employee_id
    LEFT JOIN invoice i on c.customer_id = i.customer_id
WHERE e.title = "Sales Support Agent";

Done.


[]

Created VIEW to see performance of Sales Support Agents. I am asked to measure their performance based on total sales of their customers. However, it is easire to see it for certain period, so I also add date when sales are created. Therefore, you can see their customers' sales total by using 'like [date]'. 

In [5]:
%%sql
SELECT name, hire_date, SUM(total) total
from sales_agent
GROUP BY name 

Done.


name,hire_date,total
Jane Peacock,2017-04-01 00:00:00,1731.510000000004
Margaret Park,2017-05-03 00:00:00,1584.0000000000032
Steve Johnson,2017-10-17 00:00:00,1393.9200000000028


In seeing only total column, Jane made the best performance. However, Her hire date is the earliest, so this might be due to her date when she was hired. Likewise, Steve has the worst total of sales, but his hire date is the latest. Therefore, it mighe be better to see their performance in a specific year. 

In [6]:
%%sql
SELECT name, hire_date,SUM(total) total, COUNT(date) number_of_sales
from sales_agent
WHERE date like "2019%"
group by name

Done.


name,hire_date,total,number_of_sales
Jane Peacock,2017-04-01 00:00:00,383.1300000000001,49
Margaret Park,2017-05-03 00:00:00,400.95000000000016,54
Steve Johnson,2017-10-17 00:00:00,437.57999999999987,56


This is data from 2019. Based on this, Steve has the best performance and Jane has the worst. This is a totally opposite insight to what we got earlier. 

In [7]:
%%sql
SELECT name, hire_date,SUM(total) total, COUNT(date) number_of_sales
from sales_agent
WHERE date like "2018%"
group by name

Done.


name,hire_date,total,number_of_sales
Jane Peacock,2017-04-01 00:00:00,413.81999999999994,49
Margaret Park,2017-05-03 00:00:00,400.95000000000005,53
Steve Johnson,2017-10-17 00:00:00,332.63999999999993,49


In 2018, Jane is the best. Steve is the worst. Margaret might keep making consistent results since she is always in 2nd position. Therefore, since there is variation in each year, it is a better practice to see their performance on periodic basis.

## Task 3 
In this task, I was asked to analyze the sales data for customers from each different country. Specifically, I was required to calculate following values for each country. 
- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Moreover, I was also asked to classify countries, whose number of customers is only one, as "Other". 

In [8]:
%%sql
WiTH customer_with_classifier AS
(
    SELECT 
        c.*,
        i.invoice_id invoice_id,
        i.total total,
    CASE
    WHEN
        (SELECT COUNT(customer_id)
        FROM customer
        WHERE country = c.country ) = 1
            THEN 'other'
            ELSE c.country
        END AS classification
    FROM customer c 
    INNER JOIN invoice i ON c.customer_id = i.customer_id
)

SELECT 
    classification, 
    COUNT(DISTINCT(customer_id)) total_num_custoemr,
    SUM(total) total_sales,
    CAST(SUM(total) AS float)/COUNT(DISTINCT(customer_id)) avg_per_customer,
    CAST(SUM(total) AS float)/COUNT(invoice_id) avg_per_order
FROM 
    (
        
        SELECT
        customer_with_classifier.*,
        CASE 
            WHEN
                classification = 'other'
                THEN 0
                ELSE 1
            END AS sort 
        FROM customer_with_classifier
    )
GROUP BY classification 
ORDER BY sort DESC, total_sales DESC



Done.


classification,total_num_custoemr,total_sales,avg_per_customer,avg_per_order
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
United Kingdom,3,245.52,81.84,8.768571428571429
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
India,2,183.14999999999998,91.57499999999999,8.72142857142857
other,15,1094.9400000000003,72.99600000000002,7.44857142857143


In USA, we have the largerst number of customers, so the total amount of sales is also the best. However, in seeing per customer base and order base, the secenery looks different. There are other countries whose per customer and order values are higher than USA. This could be because of the small number of customers. 
I think that the size of this data is small, so I recommend to avoid heavily relying on this data and to collect more data when the business makes business strategies. For instance, Czech has very high total value per customer, but since the number of the customers is two, it is not reliable number

## Task 4 
The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

- purchase a whole album
- purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [9]:
%%sql 
WITH invoice_and_track AS
(
    SELECT 
        invoice_id,
        track_id
    FROM invoice_line 
    GROUP BY invoice_id 
)

SELECT 
    purchase_type, 
    COUNT(invoice_id) number_of_invoice,
    CAST(COUNT(invoice_id) AS float) / (SELECT COUNT(*) FROM invoice_and_track) percentage
FROM 
(
    SELECT 
        invoice_id,
        CASE
            WHEN
                    (
                        SELECT track_id
                        FROM invoice_line
                        WHERE invoice_id = it.invoice_id

                        EXCEPT

                        SELECT track_id 
                        FROM track 
                        WHERE album_id = (SELECT album_id
                                          FROM track
                                          WHERE track_id = it.track_id) 
                    ) IS NULL 
                    
                    AND
                    
                    (
                        SELECT track_id 
                        FROM track 
                        WHERE album_id = (SELECT album_id
                                          FROM track
                                          WHERE track_id = it.track_id)
                        EXCEPT

                        SELECT track_id
                        FROM invoice_line
                        WHERE invoice_id = it.invoice_id
                    ) IS NULL    
            THEN 'album'
            ELSE 'not_album'
        END AS purchase_type
    FROM invoice_and_track it 
)
GROUP by purchase_type
ORDER BY number_of_invoice DESC





Done.


purchase_type,number_of_invoice,percentage
not_album,500,0.8143322475570033
album,114,0.18566775244299674


Based on this result, album purchase occupy about 20 % of all purchases. Therefore, I would not recommend to stop purchasing every track from an album since it leads loss of revenue, which is about 20% loss. 